In [1]:
%load_ext autoreload


In [2]:
%autoreload 2

In [4]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, int_norm, evaluation
from pylab import plot, show, savefig
from scipy import stats
#import seaborn as sns
#%pylab inline

In [5]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2016-02-17/"
path_sav = "/Volumes/friebelm/PhD/NU Plasma/Measurements/compilation/"
files_start = 3281
files_end = 3476

In [65]:
# cup configuration
cup_config = cycle_Sb_2
# Isotopes used for Interference correction
corr_isotopes_1 = {"Cd" : "111", "Te" : "125"}
corr_isotopes_2 = {"Cd" : "111","Te" : "125", "Xe" : "129"}
#corr_isotopes_Sb = {"Te": "125", "Xe" : "129"}
corr_isotopes_Sb = {}
#
# Mass Range of cup configuration
mass_range = cycle_Sb_2_mass_range
#isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"], ["117", "118", "119", "122", "124"]]
#isotopes = [["112", "114", "115", "116", "117", "118", "119", "122", "124"]]
isotopes = [["116", "117", "118", "119", "122", "124"]]
isotopes_column = ["116", "117", "118", "119", "122", "124"]
data_sample_column = [(i + "/" + "120") for i in isotopes_column]

denom_isotope = "120"
norm_ratio = ["116","120"]
norm_ratio_Sb= ["123","121"]
#number of iterations for beta
iter_beta = 10

# Interference_corr on the denominator isotope
isotope_denom_corr = False

blk_corr = False

In [60]:
columns = ["cycle", "sample", "date", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)"]
blk_ls = []
sample_ls = []
df_all = pd.DataFrame()
#files_1 = range(files_start, files_end, 1)
files_1 = range(3281, 3444) + range(3445, 3476)
for sample in files_1:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    df_zero = df.data_zero_corr(sample)
    df_zero = pd.DataFrame(df_zero["cycle1"])
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    df_zero["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    df_zero["sample"] = sample_name
    df_zero["cycle"] = df_zero.index.values
    df_zero.index = df_zero["date"]
    df_zero = df_zero[columns]
    df_all = df_all.append(df_zero, ignore_index = True)

    if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean":
        blk_ls.append(sample)
    elif sample_name == "SQ" or sample_name == "teflon blk" or sample_name == "blank 3_1" or sample_name == "col blk 271115":
        None
    else:
        sample_ls.append(sample)

df_all.set_index("date", inplace = True)
df_all

,cycle,sample,H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),Ax (1),L1 (1),L2 (1),L3 (1)
date,,,,,,,,,,,,,,
2016-02-18 15:29:00,1,blank sol,0.015104,0.015905,3.557100e-06,0.000392,0.002808,0.000254,0.003620,0.002831,0.000502,0.001381,0.000485,0.001040
2016-02-18 15:29:00,2,blank sol,0.015130,0.015927,1.493920e-05,0.000385,0.002822,0.000263,0.003682,0.002941,0.000529,0.001479,0.000489,0.001080
2016-02-18 15:29:00,3,blank sol,0.015223,0.016083,2.774400e-05,0.000468,0.002897,0.000314,0.003823,0.003153,0.000585,0.001592,0.000518,0.001176
2016-02-18 15:29:00,4,blank sol,0.015288,0.016139,4.908540e-05,0.000368,0.002805,0.000273,0.003629,0.002932,0.000521,0.001482,0.000515,0.001090
2016-02-18 15:29:00,5,blank sol,0.015172,0.016012,9.959400e-06,0.000486,0.002848,0.000308,0.003738,0.003223,0.000628,0.001683,0.000561,0.001191
2016-02-18 15:29:00,6,blank sol,0.015269,0.016062,1.422770e-05,0.000404,0.002808,0.000275,0.003644,0.002952,0.000525,0.001447,0.000495,0.001115
2016-02-18 15:29:00,7,blank sol,0.015242,0.016039,1.565030e-05,0.000394,0.002786,0.000265,0.003633,0.002839,0.000491,0.001425,0.000460,0.001033
2016-02-18 15:29:00,8,blank sol,0.015224,0.015997,3.556900e-05,0.000387,0.002776,0.000305,0.003641,0.002938,0.000530,0.001471,0.000511,0.001065
2016-02-18 15:29:00,9,blank sol,0.015218,0.016048,2.774400e-05,0.000367,0.002657,0.000245,0.003492,0.002766,0.000478,0.001325,0.000460,0.001013


In [66]:
files_1 = range(files_start, files_end, 1)
df_raw_signals = pd.DataFrame()
df_external_norm = pd.DataFrame()
df_external_norm_mean = pd.DataFrame()
df_external_outlier = pd.DataFrame()

for sample in sample_ls:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)

# arbitrary blank positions
    #blk1 = [item for item in blk_ls if item < sample]
    #blk2 = [item for item in blk_ls if item > sample]
    #blk1 = blk1[-1]
    #blk2 = blk2[0]
    #blk_corr_sample = True
    
#blank measurement surrounds sample
    #if ((sample-1) in blk_ls) and ((sample+1) in blk_ls):
    #    blk1 = sample-1
    #    blk2 = sample+1
    #    blk_corr_sample = True
    #else:
    #    blk_corr_sample = False    
    
    if (blk_corr == True) and (blk_corr_sample == True):
        blk_1 = NU_data_read(path, blk1, cup_config)
        blk_2 = NU_data_read(path, blk2, cup_config)
        df_zero = df.data_zero_corr(sample)
        df_bgd_1 = blk_1.data_zero_corr(blk1)
        df_bgd_2 = blk_2.data_zero_corr(blk2)
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_Sb , denom_corr_ratio)
        new_corr.data_bgd_corr(df_bgd_1, df_bgd_2)
    else:
        df_zero = df.data_zero_corr(sample)
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_Sb , denom_corr_ratio)
    
    data_sample_raw = new_corr.raw_signals(denom_isotope)
    data_sample_raw = pd.DataFrame.from_dict(data_sample_raw, orient = 'index')
    data_sample_raw = new_corr.mad_outlier_rejection(data_sample_raw)
    
    #data_sample = new_corr.external_norm_Sb(norm_ratio_Sb, denom_isotope, iter_beta)
    data_sample = new_corr.internal_norm_1(norm_ratio, denom_isotope, iter_beta)
# new method for df with all data together with signals in pycharm
    data_sample = pd.DataFrame.from_dict(data_sample, orient = 'index')
    data_sample = new_corr.mad_outlier_rejection(data_sample)
        
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_sample = data_sample.drop(denom_isotope, axis=1)
    data_sample = data_sample[sorted(data_sample.columns)]
    data_sample.columns = data_sample_column
    data_sample["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_sample["sample"] = sample_name
    data_sample["cycle"] = data_sample.index.values
    #data_sample_outlier = data_sample_outlier[["116", "117", "119", "120", "122", "124"]]
    #data_sample_outlier.columns = ["116/118", "117/118", "119/118", "120/118", "122/118", "124/118"]
    #data_sample_outlier["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    #data_sample_outlier["sample"] = sample_name
    #data_sample_outlier["cycle"] = data_sample.index.values
    
    
    data_sample_raw["cycle"] = data_sample_raw.index.values
    data_sample = data_sample.merge(data_sample_raw)
    
    df_external_norm = df_external_norm.append(data_sample)
    #df_external_outlier = df_external_outlier.append(data_sample_outlier) 
    print sample, sample_name

3282 NIST 100ppb
3283 NIST 100ppb
3284 NIST 100ppb
3285 NIST 100ppb
3286 NIST 100ppb
3288 NIST 100ppb
3289 NIST 100ppb
3290 NIST 100ppb
3291 NIST 100ppb
3292 NIST 100ppb
3294 NIST 100ppb
3295 SPEX 100ppb
3296 NIST 100ppb
3297 SPEX 100ppb
3298 NIST 100ppb
3300 NIST 100ppb
3301 SPEX 100ppb
3302 NIST 100ppb
3303 SPEX 100ppb
3304 NIST 100ppb
3306 NIST 100ppb
3307 J2 4_1 I
3308 NIST 100ppb
3309 ZH09-05 50.5_1(2) I
3310 NIST 100ppb
3312 NIST 100ppb
3313 ZH09-05 56.7_1(2) I
3314 NIST 100ppb
3315 ZH09-05 44.8_1(2) I
3316 NIST 100ppb
3318 NIST 100ppb
3319 NIST 100ppb
3320 NIST 100ppb
3321 NIST 100ppb
3322 NIST 100ppb
3324 NIST 100ppb
3325 ZH09-05 23.3_2_1(1) I
3326 NIST 100ppb
3327 ZH09-05 10.3_1(2) I
3328 NIST 100ppb
3330 NIST 100ppb
3331 NIST 5_1 I
3332 NIST 100ppb
3333 J2 3_1(1) I
3334 NIST 100ppb
3336 NIST 100ppb
3337 NIST 100ppb
3338 NIST 100ppb
3339 NIST 100ppb
3340 NIST 100ppb
3342 NIST 100ppb
3343 ZH09-05 50.5_1(1) I
3344 NIST 100ppb
3345 ZH09-05 56.7_1(1) I
3346 NIST 100ppb
3348 NIST 1

In [69]:
files_1 = range(files_start, files_end, 1)
df_raw_signals = pd.DataFrame()
df_external_norm = pd.DataFrame()
df_external_norm_mean = pd.DataFrame()
df_external_outlier = pd.DataFrame()

for sample in sample_ls:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    # method for blank correction
    # arbitrary blank positions
    #blk1 = [item for item in blk_ls if item < sample]
    #blk2 = [item for item in blk_ls if item > sample]
    #blk1 = blk1[-1]
    #blk2 = blk2[0]
    #blk_corr_sample == True
    
    # blank measurement surrounds sample
    #if ((sample-1) in blk_ls) and ((sample+1) in blk_ls):
    #    blk1 = sample-1
    #    blk2 = sample+1
    #    blk_corr_sample = True
    #else:
    #    blk_corr_sample = False  
    if (blk_corr == True) and (blk_corr_sample == True):
        blk_1 = NU_data_read(path, blk1, cup_config)
        blk_2 = NU_data_read(path, blk2, cup_config)
        df_zero = df.data_zero_corr(sample)
        df_bgd_1 = blk_1.data_zero_corr(blk1)
        df_bgd_2 = blk_2.data_zero_corr(blk2)
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_Sb , denom_corr_ratio)
        new_corr.data_bgd_corr(df_bgd_1, df_bgd_2)
    else:
        df_zero = df.data_zero_corr(sample)
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_Sb , denom_corr_ratio)
    
    data_sample_raw = new_corr.raw_signals("118")
    data_sample_raw = pd.DataFrame.from_dict(data_sample_raw, orient = 'index')
    data_sample_raw = data_sample_raw[np.abs(data_sample_raw-data_sample_raw.mean())<=(2*data_sample_raw.std())]
    
    data_sample = new_corr.external_norm_Sb("118", 10)
# new method for df with all data together with signals in pycharm
    data_sample = pd.DataFrame.from_dict(data_sample, orient = 'index')
    data_sample = data_sample[np.abs(data_sample-data_sample.mean())<=(2*data_sample.std())]
    data_sample_outlier= data_sample[np.abs(data_sample-data_sample.mean())>(2*data_sample.std())]
    
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_sample = data_sample[["116", "117", "119", "120", "122", "124"]]
    data_sample.columns = ["116/118", "117/118", "119/118", "120/118", "122/118", "124/118"]
    data_sample["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_sample["sample"] = sample_name
    data_sample["cycle"] = data_sample.index.values
    data_sample_outlier = data_sample_outlier[["116", "117", "119", "120", "122", "124"]]
    data_sample_outlier.columns = ["116/118", "117/118", "119/118", "120/118", "122/118", "124/118"]
    data_sample_outlier["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_sample_outlier["sample"] = sample_name
    data_sample_outlier["cycle"] = data_sample.index.values
    
    
    data_sample_raw["cycle"] = data_sample_raw.index.values
    data_sample = data_sample.merge(data_sample_raw)
    
    df_external_norm = df_external_norm.append(data_sample)
    df_external_outlier = df_external_outlier.append(data_sample_outlier) 

In [67]:
df_external_norm.set_index("date", inplace = True)
columns = ["cycle", "sample"]
columns.extend(data_sample_column)
columns.extend(sorted(data_sample_raw.drop("cycle", axis=1).columns))
df_external_sub = pd.DataFrame(df_external_norm, columns = columns)

df_all_sub = df_all[(df_all["sample"] != "blank sol") & (df_all["sample"] != "teflon blk") & (sample_name != "blank 3_1") & (sample_name != "col blk 271115") & (df_all["sample"] != "SQ")]
df_external_sub["125"] = df_all_sub["H6 (1)"]
df_external_sub["Te/Sn"] = (df_all_sub["H6 (1)"]/7.0509)/(df_all_sub["L1 (1)"]/24.223)

#df_external_sub = df_external_sub[df_external_sub["118"] > 4]
df_external_sub

,cycle,sample,116/120,117/120,118/120,119/120,122/120,124/120,116,117,118,119,120,122,124,125,Te/Sn
date,,,,,,,,,,,,,,,,,
2016-02-18 15:36:00,1,NIST 100ppb,NaN,0.235277,0.742845,0.263420,0.142028,0.177504,5.059619,2.703485,8.644518,3.104275,11.932197,1.736987,2.224133,-1.150060e-05,-4.570494e-06
2016-02-18 15:36:00,2,NIST 100ppb,NaN,0.235286,0.742895,0.263454,0.142045,0.177529,5.131566,2.741978,8.767727,3.148670,12.101041,1.761712,2.255765,-1.505760e-05,-5.900001e-06
2016-02-18 15:36:00,3,NIST 100ppb,0.446,0.235309,0.742900,0.263439,0.142041,0.177509,5.170693,2.763091,8.834240,3.172276,12.192222,1.774860,2.272312,-1.292340e-05,-5.025635e-06
2016-02-18 15:36:00,4,NIST 100ppb,0.446,0.235321,0.742949,0.263457,0.142062,0.177505,5.219220,2.789134,8.917510,3.202149,12.305999,1.791638,2.293347,-2.146010e-05,-8.267449e-06
2016-02-18 15:36:00,5,NIST 100ppb,NaN,0.235288,0.742916,0.263438,0.142036,0.177520,5.219525,2.788918,8.917723,3.202154,12.306972,1.791472,2.293765,9.129500e-06,3.517032e-06
2016-02-18 15:36:00,6,NIST 100ppb,0.446,0.235331,0.742991,0.263471,0.142058,0.177533,5.275832,2.819643,9.015595,3.237502,12.441799,1.811527,2.319447,-1.078930e-05,-4.111329e-06
2016-02-18 15:36:00,7,NIST 100ppb,0.446,0.235297,0.742871,0.263419,0.142039,0.177511,5.150475,2.752144,8.799332,3.159614,12.144466,1.767876,2.263424,-1.007770e-05,-3.934550e-06
2016-02-18 15:36:00,8,NIST 100ppb,0.446,0.235287,0.742904,0.263425,0.142053,0.177525,5.201194,2.779147,8.886508,3.190883,12.264421,1.785541,2.286030,-7.232400e-06,-2.795984e-06
2016-02-18 15:36:00,9,NIST 100ppb,0.446,0.235302,0.742934,0.263437,0.142052,0.177525,5.258688,2.810024,8.984909,3.226197,12.399469,1.805158,2.311100,-2.928520e-05,-1.119742e-05


In [68]:
def SE(x):
    SE = np.std(x) / x.count()
    return SE

df_external_norm_mean = df_external_sub.drop("cycle", axis=1).groupby(df_external_sub.index).mean()
df_external_norm_mean_sd = df_external_sub.drop("cycle", axis=1).groupby(df_external_sub.index).agg([np.mean, SE])
df_external_norm_mean["sample"] = df_external_norm[df_external_norm["cycle"] == 1]["sample"]
df_external_norm_mean_sd["sample"] = df_external_norm[df_external_norm["cycle"] == 1]["sample"]

#df_external_norm_mean = df_external_norm_mean[1:]
#df_external_norm_mean = df_external_norm_mean.drop(datetime.strptime("20150310 23:59", "%Y%m%d %H:%M"))
#df_external_norm_mean_sd = df_external_norm_mean_sd[1:]
df_external_norm_mean

,116/120,117/120,118/120,119/120,122/120,124/120,116,117,118,119,120,122,124,125,Te/Sn,sample
date,,,,,,,,,,,,,,,,
2016-02-18 15:36:00,0.446,0.235305,0.742926,0.263446,0.142050,0.177518,5.216343,2.787487,8.912806,3.200505,12.300554,1.790793,2.292748,-7.659178e-06,-2.959734e-06,NIST 100ppb
2016-02-18 15:44:00,0.446,0.235307,0.742929,0.263455,0.142054,0.177519,5.082725,2.716103,8.684521,3.118606,11.985423,1.744964,2.234014,-8.631493e-06,-3.410085e-06,NIST 100ppb
2016-02-18 15:53:00,0.446,0.235308,0.742924,0.263451,0.142054,0.177517,5.208186,2.783133,8.898682,3.195471,12.280870,1.787946,2.288975,-1.141173e-05,-4.398011e-06,NIST 100ppb
2016-02-18 16:02:00,0.446,0.235308,0.742935,0.263455,0.142057,0.177518,5.273282,2.817903,9.009928,3.235410,12.434087,1.810268,2.317506,-8.987152e-06,-3.408576e-06,NIST 100ppb
2016-02-18 16:11:00,0.446,0.235310,0.742929,0.263453,0.142055,0.177513,5.062526,2.705322,8.649867,3.106124,11.937397,1.737952,2.224908,-6.479500e-06,-2.584795e-06,NIST 100ppb
2016-02-18 16:26:00,0.446,0.235310,0.742931,0.263455,0.142056,0.177514,5.057127,2.702313,8.639936,3.102424,11.922676,1.735678,2.221816,-1.019585e-06,-4.098651e-07,NIST 100ppb
2016-02-18 16:35:00,0.446,0.235313,0.742928,0.263455,0.142056,0.177516,5.105499,2.728298,8.723146,3.132452,12.038379,1.752638,2.243712,-1.339755e-06,-5.345239e-07,NIST 100ppb
2016-02-18 16:44:00,0.446,0.235312,0.742932,0.263459,0.142057,0.177515,5.284500,2.823929,9.028924,3.242247,12.460023,1.814011,2.322211,-5.264272e-06,-2.007379e-06,NIST 100ppb
2016-02-18 16:52:00,0.446,0.235311,0.742925,0.263455,0.142056,0.177514,5.076658,2.712810,8.673538,3.114597,11.969574,1.742570,2.230713,-1.464272e-06,-5.593109e-07,NIST 100ppb


In [11]:
plot_Nist = df_external_norm_mean[df_external_norm_mean["sample"] == "NIST 100ppb"]
plot_Nist = df_external_norm_mean[(df_external_norm_mean["118"] >=1)]
#dataplot = plot(plot_Nist["Te/Sn"], plot_Nist["124/120"], "ro")
dataplot = plot(plot_Nist.index, plot_Nist["122/120"], "ro")
show(dataplot)

In [36]:
df_external_norm_mean = df_external_norm_mean[(df_external_norm_mean["118"] >=1)]
sub_index = df_external_norm_mean[(df_external_norm_mean["sample"].shift(1) == df_external_norm_mean["sample"].shift(-1)) & (df_external_norm_mean["sample"].shift(1) == "NIST 100ppb")].index
sample = df_external_norm_mean["sample"]
new = df_external_norm_mean[df_external_norm_mean.columns[df_external_norm_mean.dtypes != 'object']]

new =  ((new / ((new.shift(1) + new.shift(-1))/2))-1)*10000
new["sample"] = sample

new = new[new.index.isin(sub_index)]
#df_external_norm_signal = df_external_norm_signal[df_external_norm_signal['sample'].isin(sub_index.tolist())]
#df_new_signal = new.merge(df_external_norm_signal, left_index=True, right_index=True)
#new[new.sample != "NIST 100ppb"]
new

,116/120,117/120,118/120,119/120,122/120,124/120,116,117,118,119,120,122,124,sample
date,,,,,,,,,,,,,,
2015-12-20 11:27:00,0.510819,0.617528,0.207144,0.202255,0,-0.376816,-121.922254,-121.602910,-121.723345,-121.460420,-121.445235,-128.663972,-120.886616,NIST 100ppb
2015-12-20 11:36:00,0.165828,-0.149057,0.084055,0.128361,0,-0.106977,245.173226,245.343001,245.724464,246.062976,246.271298,246.855143,247.390345,NIST 100ppb
2015-12-20 11:45:00,-0.251718,0.150390,-0.187216,-0.128040,0,0.119019,-61.694545,-61.892051,-62.406281,-62.728524,-62.991176,-63.714299,-64.354965,NIST 100ppb
2015-12-20 11:53:00,0.440232,0.228823,0.248346,0.234389,0,-0.291870,-265.049540,-265.095498,-265.057563,-264.969368,-265.120415,-264.989393,-265.081632,NIST 100ppb
2015-12-20 12:02:00,-0.099769,-0.137819,-0.040081,-0.099030,0,0.110962,237.207713,237.137292,237.165185,237.043132,242.599085,237.080219,242.564694,NIST 100ppb
2015-12-20 12:17:00,-0.045378,0.104162,-0.022023,-0.021224,0,-0.108528,-28.442931,-28.257265,-28.311248,-28.262254,-38.676552,-28.135665,-38.630269,NIST 100ppb
2015-12-20 12:26:00,-0.309705,-0.389556,-0.199550,-0.079882,0,0.355936,122.997524,122.896842,123.105208,123.276100,128.752471,123.440590,129.268345,NIST 100ppb
2015-12-20 12:35:00,0.508756,0.464390,0.270532,0.149154,0,-0.423253,-287.462446,-287.366467,-287.531128,-287.648472,-287.754767,-287.715220,-288.108843,NIST 100ppb
2015-12-20 12:43:00,-0.199842,-0.140055,-0.069444,-0.084084,0,0.296724,259.838543,260.090640,260.352971,260.529247,260.793294,261.156165,261.823429,NIST 100ppb


In [37]:
def SD2(x):
    SD2 = np.std(x) * 2
    return SD2

#new = new.drop(["116", "117", "118", "119", "120", "122", "124"], axis=1)
np.round(new.groupby("sample").agg([np.mean, SD2]), decimals=2)

116/120       117/120       118/120       119/120        \
                        mean   SD2    mean   SD2    mean   SD2    mean   SD2   
sample                                                                         
J2 2_1(2)               0.20  0.00   -0.13  0.00   -0.02  0.00    0.13  0.00   
J2 3_1(1)              -6.07  0.00   -6.13  0.00   -4.73  0.00   -2.26  0.00   
Jenny 12-06W 57_1(1)   -0.16  0.00   -0.25  0.00    0.06  0.00   -0.05  0.00   
NIST 100ppb             0.02  0.56    0.01  0.51    0.00  0.28    0.01  0.29   
NIST 3_1               -0.15  0.00   -0.17  0.00   -0.03  0.00    0.10  0.00   
NIST 4_1                0.08  0.00    0.10  0.00    0.02  0.00   -0.01  0.00   
SPEX 100ppb            -0.17  0.73   -0.10  0.60   -0.07  0.41   -0.08  0.30   
ZH09-05 10.3_1(1)      -0.06  0.00   -0.22  0.00    0.05  0.00   -0.16  0.00   
ZH09-05 23.3_1_1(1)     3.27  0.00   -0.17  0.00    0.04  0.00   -0.12  0.00   
ZH09-05 23.3_1_2(1)     0.47  0.00   -0.00  0.00    0.06  0.00   -0.06  0.00   
ZH09-05 33.8_1(1)       0.51  0.00   -0.10  0.00    0.07  0.00    0.06  0.00   
ZH09-05 44.8_1(1)       0.03  0.00   -0.00  0.00    0.07  0.00   -0.03  0.00   
ZH09-05 50.5_1(1)      -0.28  0.00   -0.38  0.00   -0.02  0.00   -0.03  0.00   
ZH09-05 56.7_1(1)      -0.33  0.00   -0.36  0.00   -0.10  0.00   -0.04  0.00   

                     122/120       ...        118              119          \
                        mean SD2   ...       mean     SD2     mean     SD2   
sample                             ...                                       
J2 2_1(2)                  0   0   ...     642.73    0.00   643.74    0.00   
J2 3_1(1)                  0   0   ...   -7958.50    0.00 -7958.07    0.00   
Jenny 12-06W 57_1(1)       0   0   ...     898.92    0.00   899.41    0.00   
NIST 100ppb                0   0   ...      -6.10  352.01    -6.21  351.58   
NIST 3_1                   0   0   ...     924.77    0.00   924.69    0.00   
NIST 4_1                   0   0   ...     859.77    0.00   858.88    0.00   
SPEX 100ppb                0   0   ...      52.85  359.93    53.27  361.19   
ZH09-05 10.3_1(1)          0   0   ...     939.29    0.00   941.93    0.00   
ZH09-05 23.3_1_1(1)        0   0   ...     978.70    0.00   986.36    0.00   
ZH09-05 23.3_1_2(1)        0   0   ...     825.47    0.00   825.31    0.00   
ZH09-05 33.8_1(1)          0   0   ...    1095.53    0.00  1095.95    0.00   
ZH09-05 44.8_1(1)          0   0   ...     749.95    0.00   748.65    0.00   
ZH09-05 50.5_1(1)          0   0   ...    1959.29    0.00  1958.85    0.00   
ZH09-05 56.7_1(1)          0   0   ...     947.34    0.00   947.97    0.00   

                          120              122              124          
                         mean     SD2     mean     SD2     mean     SD2  
sample                                                                   
J2 2_1(2)              644.46    0.00   646.13    0.00   647.78    0.00  
J2 3_1(1)            -7957.64    0.00 -7957.73    0.00 -7957.00    0.00  
Jenny 12-06W 57_1(1)   900.09    0.00   901.42    0.00   902.63    0.00  
NIST 100ppb             -6.02  352.60    -6.15  352.08    -6.04  352.79  
NIST 3_1               924.36    0.00   923.92    0.00   923.51    0.00  
NIST 4_1               858.04    0.00   856.36    0.00   854.59    0.00  
SPEX 100ppb             54.39  360.33    56.63  360.30    57.69  361.75  
ZH09-05 10.3_1(1)      944.91    0.00   950.51    0.00   956.17    0.00  
ZH09-05 23.3_1_1(1)    994.25    0.00  1009.61    0.00  1024.79    0.00  
ZH09-05 23.3_1_2(1)    825.31    0.00   825.18    0.00   824.86    0.00  
ZH09-05 33.8_1(1)     1096.28    0.00  1097.13    0.00  1097.58    0.00  
ZH09-05 44.8_1(1)      747.59    0.00   745.41    0.00   742.97    0.00  
ZH09-05 50.5_1(1)     1958.45    0.00  1957.66    0.00  1951.36    0.00  
ZH09-05 56.7_1(1)      948.59    0.00   949.70    0.00   951.00    0.00  

[14 rows x 26 columns]

In [33]:
df_all.to_csv(path + "Sn_H8_L3_Sb_NU_Signals_zero_corr.csv", mode='w', header=True)

In [100]:
df_external_sub.to_csv(path_sav + "Sn_H6_L3_SB_external_norm.csv", mode='a', header=False)

In [69]:
df_external_norm_mean_sd.to_csv(path + "Sn_H8_L3_SB_internal_norm_116_120_MAD.csv", mode='w', header=True)